## **HW3 Part 2**

Yash Maniya (B20CS033)

In [2]:
import random
import hashlib
from Crypto.Util.number import getPrime

(a) An RSA-PKCS#1 encryption implementation which can be given any arbitrary message
m to encrypt and produce the ciphertext c. You will call this to encrypt a message
created by you, which should include your roll number. Rest of the message can be
anything but at least some part of the message should be randomly chosen every time
this function is called. (Your code should NOT be tailored to work on a specific message
only).

In [40]:
def pkcs1_pad(m, n):
    m = hashlib.sha256(m.encode()).digest()
    
    # Calculate the size of the modulus in bytes
    k = (n.bit_length() + 7) // 8
    
    # Maximum size of the padding
    t = k - len(m) - 3
    
    # Generate non-zero random padding
    ps = bytearray(random.getrandbits(8) for i in range(t))
    while 0 in ps:
        ps[ps.index(0)] = random.getrandbits(8)

    # Construct the padded message
    pm = bytearray([0x00, 0x02]) + ps + bytearray([0x00]) + m
    
    return pm

def rsa_encrypt(m, e, n, flag=False):
    pm = pkcs1_pad(m, n)
    if flag: 
        print(f'Padded Message \t\t: {pm}\n')
    m_int = int.from_bytes(pm, 'big')
    c_int = pow(m_int, e, n)
    c = c_int.to_bytes((c_int.bit_length() + 7) // 8, 'big')
    
    return c
    

In [47]:
roll_number = "B20CS033"
m = f"Is you roll number {roll_number} ?"
e = 65537
n = getPrime(1024)

print(f'Message \t\t: {m}\n')

ciphertext = rsa_encrypt(m, e, n, True)

print(f'Ciphertext \t: {ciphertext}')


Message 		: Is you roll number B20CS033 ?

Padded Message 		: bytearray(b"\x00\x02\x0b\x8dW\xc4\xf8\x07i\xc5\x8f\xa9\x8c\x1b\x0be\x05\xac\n3\'\xf6\xa8\xcb\xf6j\xc6H\xb5\x84\xba5A\xbd\x86F\xea\xb4oi\xfb%\xde\xa2\x07\xda\x01\xe2O\x88\x0b\xed4\n\xe7%\xa2D\xd0\x8dHi\xbe\xc6\xec^\xe1t\xb6\xa3}\xed\x94\x19\xba\xd6i\x1c\x15\x9c\x94T\x98\xa4v\x8e<C\x93R[\xf4f{\xc4\x007\xc7Wf\x90|\xc4\x98N\x80\xfc\xdax\x8eK\xfd\x84\xc0\xcen\xfaO\x15@ \x87\xa9\xcf\x14s\xaa&")

Ciphertext 	: b'\x08\x86\xdd_\xa9\xd7+\xf9\x07\x89\x84\xa4\xbf\x83\x0fqT\xae\x8c\xe7\xf9a\xab\x85\xd3}\xd5\xfep\x00w\xb1\xdb\x00\xf7\xbdC\xf1\xab>\xd3,\x13\x97\xad\xder\x07\x99\xd6\xdeU7\xc8\x9fO\xe6^\x14\x05\xbe#\xb4t\xbax\xfc3\xed\x9eIW\xe02J\x15\x93;id\t\x7f\xc6\x86\xc8\xde\xc2\xad\xfc\x95\xc3\x99@-\x94\xa7\x82\x88\xf6f#\xd0\xf3\xad\x1d\x84$\xad\xb6pW\xfe0\xa8\x7f\xb5j\xfe\xdf\x19`\x81\\B\xfcU\xb40'


(b) A decryption oracle which returns a single bit answer informing if the decrypted message
is PKCS#1 conforming.

In [38]:
def oracle(pm):
    # Verify that the padding format is correct
    if pm[0] != 0x00 or pm[1] != 0x02:
        return False
    
    # Find the end of the padding
    i = pm.find(0x00, 2)
    if i == -1:
        return False
    
    # Verify that the padding is non-zero and has the correct size
    ps = pm[2:i]
    if len(ps) < 8:
        return False
    
    # Verify that the message is not empty
    d = pm[i+1:]
    if len(d) == 0:
        return False
    
    # The message is PKCS#1 conforming
    return True


In [39]:
test_messages = ["This is a test message with roll no. B20CS033",
                 "B20CS033",
                 "Hello world! B20CS033",
                 "The quick brown fox jumps over the lazy dog B20CS033",
                 "1234567890!@#$%^&*()_+ B20CS033"]

for msg in test_messages:
    pm = pkcs1_pad(msg, n)
    res = oracle(pm)
    
    print(f'Decrypted Message is PKCS#1 conforming : {res}')


Decrypted Message is PKCS#1 conforming : True
Decrypted Message is PKCS#1 conforming : True
Decrypted Message is PKCS#1 conforming : True
Decrypted Message is PKCS#1 conforming : True
Decrypted Message is PKCS#1 conforming : True


In [44]:
z = rsa_encrypt(test_messages[3], e, n)
print(f'Decrypted Message is PKCS#1 conforming : {oracle(z)}')

Decrypted Message is PKCS#1 conforming : False


(c) An attack function which gets the ciphertext c and can call the decryption oracle as
many times as needed (except on the ciphertext c).